In [2]:
import pandas as pd

In [2]:
from bs4 import BeautifulSoup as BS

In [3]:
from time import sleep

In [4]:
from selenium import webdriver

In [3]:
import sqlite3 as sql

In [6]:
urls = []
product_urls = []
list_of_reviews = []

In [7]:
#Each page page_url

In [9]:
for i in range(1,252):
    urls.append(f"https://www.etsy.com/in-en/c/jewelry/earrings/ear-jackets-and-climbers?ref=pagination&explicit=1&page={i}")

In [10]:
# Scrapping each product's urls | 16,064 products

In [11]:
for url in urls:
    driver = webdriver.Chrome(executable_path="/Users/nallu/chromedriver")
    driver.get(url)
    sleep(5)
    for i in range(1, 65):
        product = driver.find_element_by_xpath(f'//*[@id="content"]/div/div[1]/div/div[3]/div[2]/div[2]/div[2]/div/div/ul/li[{i}]/div/a')
        product_urls.append(product.get_attribute('href'))

In [12]:
# Scrapping each product's reviews

In [14]:
driver = webdriver.Chrome(executable_path='/Users/nallu/chromedriver')  
for product_url in product_urls[1:]:
    try:
        driver.get(product_url)
        sleep(5)
        html = driver.page_source
        soup = BeautifulSoup(html,'html')
        for i in range(4):
            try:
                list_of_reviews.append(soup.select(f'#review-preview-toggle-{i}')[0].getText().strip())
            except:
                continue
        while(True):
            try:
                next_button = driver.find_element_by_xpath('//*[@id="reviews"]/div[2]/nav/ul/li[position() = last()]/a[contains(@href, "https")]')
                if next_button != None:
                    next_button.click()
                    sleep(5)
                    html = driver.page_source
                    soup = BeautifulSoup(html,'html')
                    for i in range(4):
                        try: 
                            list_of_reviews.append(soup.select(f'#review-preview-toggle-{i}')[0].getText().strip())
                        except:
                            continue
            except Exception as e:
                print('finsish : ', e)
                break
    except:
        continue



In [18]:
scrappedReviewsAll = pd.DataFrame(list_of_reviews, index = None, columns = ['reviews'])

In [25]:
scrappedReviewsAll.to_csv('scrappedReviewsAll.csv')

In [4]:
df = pd.read_csv('scrappedReviewsAll.csv')

In [5]:
conn = sql.connect('scrappedReviewsAll.db')

In [8]:
df.to_sql('scrappedReviewsAlldb', conn) 